In [99]:
import os
import sys
import glob
import itertools

import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [100]:
print(np.__version__, sns.__version__)

1.24.3 0.12.2


In [101]:
TEST_ROOT = '../data/003_Test/img/'

In [109]:
SUBMIT_ROOTS = sorted(glob.glob('../submits/*'))
display(SUBMIT_ROOTS)

['../submits/clean-pos-idaho_neg-idaho_th0.4',
 '../submits/clean-pos_neg_th0.4',
 '../submits/idaho-sia-neg-idaho_th0.3',
 '../submits/idaho-sia-neg-idaho_th0.4',
 '../submits/idaho_neg-idaho_th0.4',
 '../submits/neg_th0.4',
 '../submits/pos',
 '../submits/pos_neg_th0.4',
 '../submits/pos_th0.4',
 '../submits/sia_pos-idaho_th0.3',
 '../submits/up16_neg-idaho_th0.3',
 '../submits/v14-v15_th0.4',
 '../submits/v64-v15_th0.4',
 '../submits/v6_v16_v18-v15_v17_th0.4_cloud-postTrue',
 '../submits/v6_v16_v18-v15_v17_th0.4_cloud-postTrue_no-cloudTrue',
 '../submits/v6_v64_v14-v15_v17_th0.4']

In [110]:
n_sub = len(SUBMIT_ROOTS)
print(n_sub)

16


# visualize inferences on test data

In [67]:
# colors = np.array([4,3,2])  # True color
colors = np.array([3,7,2])  # Natural color

In [68]:
test_imgs = sorted(glob.glob(TEST_ROOT+'*.npy'))
print(len(test_imgs))

68


In [69]:
print(len(SUBMIT_ROOTS))

10


In [70]:
PANEL_SIZE = 3

## Cloud threshold

In [ ]:
n_cols = 9
n_rows = len(test_imgs)
print(n_cols, n_rows, len(test_imgs))

fig, axes = plt.subplots(n_rows,n_cols,figsize=(PANEL_SIZE*n_cols,PANEL_SIZE*n_rows), tight_layout=True)
axes = axes.ravel()
i = 0
for j, test_img in enumerate(test_imgs):
    img = np.load(test_img)
    axes[i].imshow((img[:,:,colors+12].clip(0,5000)/5000*255).astype(np.uint8))
    axes[i].axis('off')
    i+=1
    axes[i].imshow((img[:,:,colors].clip(0,5000)/5000*255).astype(np.uint8))
    axes[i].axis('off')
    i+=1

    cloud_path = test_img.replace('../data/003_Test/img/',os.path.join(SUBMIT_ROOTS[0],'visualize/')).replace('.npy','_cloud.png')
    cloud = cv2.imread(cloud_path,0)/255.
    axes[i].imshow(cloud)
    axes[i].axis('off')
    if j==0:
        axes[i].set_title('cloud_prob')
    i+=1
    for r in [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
        axes[i].imshow(cloud>r)
        axes[i].axis('off')
        if j==0:
            axes[i].set_title(f'thr={r}')
        i+=1
plt.show()

## Compare models

In [ ]:
n_cols = 10#n_sub + 3
n_rows = len(test_imgs) * 2
print(n_cols, n_rows, len(test_imgs))

fig, axes = plt.subplots(n_rows,n_cols,figsize=(PANEL_SIZE*n_cols,PANEL_SIZE*n_rows), tight_layout=True)
axes = axes.ravel()
i = 0
for j, test_img in enumerate(test_imgs):
    img = np.load(test_img)
    axes[i].imshow((img[:,:,colors+12].clip(0,5000)/5000*255).astype(np.uint8))
    axes[i].axis('off')
    i+=1
    axes[i].imshow((img[:,:,colors].clip(0,5000)/5000*255).astype(np.uint8))
    axes[i].axis('off')
    i+=1

    for k, submit_root in enumerate(SUBMIT_ROOTS):
        if k==0:
            cloud_path = test_img.replace('../data/003_Test/img/',os.path.join(submit_root,'visualize/')).replace('.npy','_cloud.png')
            cloud = cv2.imread(cloud_path,0)
            axes[i].imshow(cloud>0.8)
            axes[i].axis('off')
            # if j==0:
            if True:
                axes[i].set_title('cloud')
            i+=1
        try:
            fire_path = test_img.replace('../data/003_Test/img/',os.path.join(submit_root,'visualize/')).replace('.npy','_fire_sup.png')
            fire = cv2.imread(fire_path,0)
            axes[i].imshow(fire, vmin=0, vmax=1)
            axes[i].axis('off')
            # if j==0:
            if True:
                axes[i].set_title(submit_root.split('/')[2])
            i+=1
        except Exception as e:
            i+=1
    i+=1        
plt.show()

## Model steps

In [ ]:
n_cols = 6
n_rows = len(test_imgs)
print(n_cols, n_rows, len(test_imgs))
submit_root = SUBMIT_ROOTS[-1]
print(submit_root)

fig, axes = plt.subplots(n_rows,n_cols,figsize=(PANEL_SIZE*n_cols,PANEL_SIZE*n_rows), tight_layout=True)
axes = axes.ravel()
i = 0
for j, test_img in enumerate(test_imgs):
    img = np.load(test_img)
    axes[i].imshow((img[:,:,colors+12].clip(0,5000)/5000*255).astype(np.uint8))
    axes[i].axis('off')
    i+=1
    axes[i].imshow((img[:,:,colors].clip(0,5000)/5000*255).astype(np.uint8))
    axes[i].axis('off')
    i+=1

    cloud_path = test_img.replace('../data/003_Test/img/',os.path.join(submit_root,'visualize/')).replace('.npy','_cloud.png')
    cloud = cv2.imread(cloud_path,0)
    axes[i].imshow(cloud, vmin=0, vmax=255)
    axes[i].axis('off')
    if j==0:
        axes[i].set_title('cloud')
    i+=1

    fire_path = test_img.replace('../data/003_Test/img/',os.path.join(submit_root,'visualize/')).replace('.npy','_fire.png')
    fire = cv2.imread(fire_path,0)
    axes[i].imshow(fire, vmin=0, vmax=255)
    axes[i].axis('off')
    if j==0:
        axes[i].set_title('fire')
    i+=1

    bin_path = test_img.replace('../data/003_Test/img/',os.path.join(submit_root,'visualize/')).replace('.npy','_fire_bin.png')
    fire_bin = cv2.imread(bin_path,0)
    axes[i].imshow(fire_bin, vmin=0, vmax=1)
    axes[i].axis('off')
    if j==0:
        axes[i].set_title('fire_bin')
    i+=1
            
    sup_path = test_img.replace('../data/003_Test/img/',os.path.join(submit_root,'visualize/')).replace('.npy','_fire_sup.png')
    fire_sup = cv2.imread(sup_path,0)
    axes[i].imshow(fire_sup, vmin=0, vmax=1)
    axes[i].axis('off')
    if j==0:
        axes[i].set_title('fire_sup')
    i+=1
            
plt.show()